In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [11]:
train = pd.read_csv('../data/preprocess/train_t1.csv')
test = pd.read_csv('../data/preprocess/test1.csv')

/var/folders/yt/n9_2kr5s6nx_5b2y_kx3mnnm0000gn/T/ipykernel_52740/390627105.py:1: DtypeWarning: Columns (6,7,8,9,11,12,13,14,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../data/preprocess/train_t1.csv')


In [12]:
train.info(), test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722815 entries, 0 to 722814
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0.1                    722815 non-null  int64  
 1   Unnamed: 0                      722815 non-null  int64  
 2   floors_before_eq (total)        390009 non-null  float64
 3   old_building                    483611 non-null  float64
 4   plinth_area (ft^2)              301607 non-null  float64
 5   height_before_eq (ft)           390009 non-null  float64
 6   land_surface_condition          421209 non-null  object 
 7   type_of_foundation              483611 non-null  object 
 8   type_of_roof                    301607 non-null  object 
 9   type_of_ground_floor            390009 non-null  object 
 10  type_of_other_floor             722815 non-null  object 
 11  position                        410809 non-null  object 
 12  building_plan_co

(None, None)

In [13]:
train = train.iloc[:,2:]
test = test.iloc[:,2:]

In [14]:
train = train.copy().drop(labels='technical_solution_proposed',axis=1)

In [15]:
train = train.copy().dropna()

In [16]:
train.shape[0]

301607

In [57]:
from catboost import CatBoostClassifier, Pool


In [73]:
from sklearn.tree import DecisionTreeClassifier
from catboost.utils import eval_metric
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score

X = train.iloc[:,:-1]
y = train.iloc[:,-1]
cat_features = X .select_dtypes(include='object').columns.tolist()
one_hot_encoded = pd.get_dummies(X[cat_features])
cols_var = [i for i in X if i not in cat_features]
df_encoded = pd.concat([X.loc[:,cols_var], one_hot_encoded], axis=1)


# Set up the StratifiedKFold for cross validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
dt = DecisionTreeClassifier()
# Loop through each split and fit the LightGBM model
for train_index, test_index in skf.split(df_encoded, y):
    X_train, X_test = df_encoded.iloc[train_index], df_encoded.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    f1_macro = f1_score(y_test, y_pred, average = 'macro')
    print(f1_macro)


0.3196757123595272
0.3193689239879962
0.31901067768293795
0.3189294053309947
0.3164401872304565
